In [20]:
import fiona

with fiona.open('Geospatial/abila.shp', 'r') as source:

    # Copy the source schema and open a sink for writing.
    sink_schema = source.schema.copy()
    with fiona.open('abila_clean.shp', 'w',
                    crs=source.crs,
                    driver=source.driver,
                    schema=sink_schema,
                    ) as sink:

        # Iterate over features in source
        for f in source:
            try:
                # Check if the geometry has more than one point
                if len(f['geometry']['coordinates']) > 1:
                    sink.write(f)
            except Exception as e:
                # If an error occurs, print it and continue with the next feature
                print("Error processing feature %s: %s" % (f['id'], e))


In [6]:
import geopandas as gpd
import pandas as pd
import plotly.graph_objects as go

# Load data
abila_map = gpd.read_file('abila_clean.shp')
gps_data = pd.read_csv('gps.csv')

# Create scatter plot for the shapefile (converting the geometry to longitude and latitude)
shapefile_scatter = go.Scattermapbox(
    lon=abila_map.geometry.centroid.x,
    lat=abila_map.geometry.centroid.y,
    mode='markers',
    marker=dict(size=5, color='blue'),  # Adjust size and color as needed
    name='Shapefile'
)

# Create scatter plot for the GPS data
gps_scatter = go.Scattermapbox(
    lon=gps_data['long'],
    lat=gps_data['lat'],
    mode='markers',
    marker=dict(size=5, color='red'),  # Adjust size and color as needed
    name='GPS'
)

# Create the layout
layout = go.Layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken='pk.eyJ1Ijoic2lhbGE5NCIsImEiOiJjbGkwZHFraHYwMnlqM2ZxdDMwcmZ0eHpqIn0.hSlCyOSAegQISYuMg98rOA',  # Replace with your Mapbox access token
        bearing=0,
        center=dict(
            lat=abila_map.geometry.centroid.y.mean(),
            lon=abila_map.geometry.centroid.x.mean()
        ),
        pitch=0,
        zoom=5,  # Adjust zoom level as needed
    ),
)

# Combine the plots
data = [shapefile_scatter, gps_scatter]

fig = go.Figure(data=data, layout=layout)
fig.show()


/var/folders/7_/cwxkqgrs0192k_wlj6m516tc0000gn/T/ipykernel_28983/2919736272.py:7: DtypeWarning:

Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.



AttributeError: module 'plotly.graph_objects' has no attribute 'line_mapbox'

In [7]:
import geopandas as gpd
import pandas as pd
import plotly.graph_objects as go

# Load data
abila_map = gpd.read_file('abila_clean.shp')
gps_data = pd.read_csv('gps.csv')

# Create scatter plot for the shapefile (converting the geometry to longitude and latitude)
shapefile_lines = go.Scattermapbox(
    lon=abila_map.geometry.apply(lambda geom: [point.x for point in geom.coords]),
    lat=abila_map.geometry.apply(lambda geom: [point.y for point in geom.coords]),
    mode='lines',
    line=dict(color='blue', width=2),  # Adjust color and width as needed
    name='Shapefile'
)

# Create scatter plot for the GPS data
gps_scatter = go.Scattermapbox(
    lon=gps_data['long'],
    lat=gps_data['lat'],
    mode='markers',
    marker=dict(size=5, color='red'),  # Adjust size and color as needed
    name='GPS'
)

# Create the layout
layout = go.Layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken='pk.eyJ1Ijoic2lhbGE5NCIsImEiOiJjbGkwZHFraHYwMnlqM2ZxdDMwcmZ0eHpqIn0.hSlCyOSAegQISYuMg98rOA',  # Replace with your Mapbox access token
        bearing=0,
        center=dict(
            lat=abila_map.geometry.centroid.y.mean(),
            lon=abila_map.geometry.centroid.x.mean()
        ),
        pitch=0,
        zoom=5,  # Adjust zoom level as needed
    ),
)

# Combine the plots
data = [shapefile_lines, gps_scatter]

fig = go.Figure(data=data, layout=layout)
fig.show()


/var/folders/7_/cwxkqgrs0192k_wlj6m516tc0000gn/T/ipykernel_28983/8208043.py:7: DtypeWarning:

Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.



AttributeError: 'tuple' object has no attribute 'x'